In [1]:
print("Praise the Lord")

Praise the Lord


In [31]:
# !pip install voila

In [8]:
import pandas as pd
import numpy as np
import joblib
import io
import warnings
warnings.filterwarnings('ignore')

In [9]:
# inp_df = pd.read_excel('./content/Sample_Input.xlsx')

In [38]:
from ipywidgets import FileUpload, Label, Layout, HBox, Button, Output, VBox
from IPython.display import display, clear_output

In [30]:
# We need to enable the extensions to be properly displayed on the notebook:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

Enabling: voila
- Writing config: D:\Rajesh\anaconda3\etc\jupyter
    - Validating...
      voila 0.5.5 ok


In [27]:
y = Label("Upload the input file using the button below", layout=Layout(display="flex", justify_content="center", width="60%", border="solid"))
display(HBox([y]))

In [5]:
uploader = FileUpload(accept='.xlsx', multiple=False)
display(uploader)


FileUpload(value=(), accept='.xlsx', description='Upload')

In [45]:
button_send = Button(
                description='Make Prediction',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = Output()

def on_button_clicked(event):
    with output:
        clear_output()
        res=make_prediction()
        print(res)
        print("Button Clicked")

button_send.on_click(on_button_clicked)
vbox_result = VBox([button_send, output])
display(vbox_result)

# display(button_send)

In [40]:
def make_prediction():
    # Read the selected features file
    data_df = pd.read_csv('./content/only_selected_features.csv')
    
    # Read the input file
    input_file = uploader.value[0]
    content = input_file['content']
    content = io.BytesIO(content)
    inp_df = pd.read_excel(content)
    
    # Merge input file with the features file and select only the required columns
    inp_merge_df = inp_df.merge(data_df,how='left',on=['Gene Name', 'Site'])
    sel_cols = ['Ht_normalize_site_value', 'Hp_normalize', 'Bl_normalize_site_value', 'Rf_normalize_site_value', 'aM_normalize',
    's_normalize_site_value', 'GRAR740102_normalize_site_value', 'ISOY800102_normalize', 'JUKT750101_normalize_site_value',
    'JUKT750101_normalize', 'KARP850101_normalize_site_value', 'MAXF760106_normalize_site_value', 
    'RACS820113_normalize_site_value', 'FASG760102_normalize_site_value', 'residue exposure', 'KABAT', 'JORES',
    'TAYLOR_NO_GAPS', 'VELIBIL', 'KARLIN', 'pssm_score2', 'conservation', 'S_containing', 'aliphatic', 'M_c_odds_ratio',
    'tri_odds_ratio']
    model_inp_df = inp_merge_df[sel_cols] 
    
    # Load the model
    clf = joblib.load('canpred_model')
    
    # Get the prediction probabilities
    pred_res = pd.DataFrame(clf.predict_proba(model_inp_df)[:,1])
    
    # Derive prediction results
    pred_res.rename(columns={0: 'Probability'}, inplace=True)
    pred_res['Prediction'] = ['Disease' if x > 0.5 else 'Neutral' for x in pred_res['Probability']]
    pred_res['Seq'] = np.arange(1, pred_res.shape[0] + 1)
    inp_df['Seq'] = np.arange(1, inp_df.shape[0] + 1)
    outp_df = inp_df.merge(pred_res,how='left')
    outp_df.drop(columns=['Seq'], inplace=True)
    
    # Output the prediction results
    return(outp_df)

In [19]:
# outp_df.to_excel('.\content\Sample_Output.xlsx',index=False)

In [46]:
print("Hallelujah!")

Hallelujah!
